In [1]:
import tensorflow as tf
import numpy as np

from Transformer import Transformer


2022-10-03 11:32:11.119171: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-10-03 11:32:11.119187: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [5]:
input_embedding = [[
  "Salut", "comment", "ca", "va", "?"
]]


output_embedding = [[
    "<START>", "Hi", "how", "are", "you", "?"
]]

def get_vocabulary(sequences):

  token_to_info = {}

  for sequence in sequences:
    for word in sequence:
      if word not in token_to_info:
        token_to_info[word] = len(token_to_info)
  
  return token_to_info

input_voc = get_vocabulary(input_embedding)
output_voc = get_vocabulary(output_embedding)

input_voc["<START>"] = len(input_voc)
input_voc["<END>"] = len(input_voc)
input_voc["<PAD>"] = len(input_voc)

output_voc["<END>"] = len(output_voc)
output_voc["<PAD>"] = len(output_voc)

print(input_voc)
print(output_voc)

def sequences_to_int(sequences, voc):
  for sequence in sequences:
    print(sequence)
    for s, word in enumerate(sequence):
      sequence[s] = voc[word]
  return np.array(sequences)

input_seq = sequences_to_int(input_embedding, input_voc)
output_seq = sequences_to_int(output_embedding, output_voc)

print("input_seq", input_seq)
print("output_seq", output_seq)

{'Salut': 0, 'comment': 1, 'ca': 2, 'va': 3, '?': 4, '<START>': 5, '<END>': 6, '<PAD>': 7}
{'<START>': 0, 'Hi': 1, 'how': 2, 'are': 3, 'you': 4, '?': 5, '<END>': 6, '<PAD>': 7}
['Salut', 'comment', 'ca', 'va', '?']
['<START>', 'Hi', 'how', 'are', 'you', '?']
input_seq [[0 1 2 3 4]]
output_seq [[0 1 2 3 4 5]]


In [7]:
transformer = Transformer(coding_block=6, dim=256, nbr_heads=8)
output = transformer((input_seq, output_seq))

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_3 (InputLayer)           [(None, 5)]          0           []                               
                                                                                                  
 input_4 (InputLayer)           [(None, 6)]          0           []                               
                                                                                                  
 embedding__layer_2 (Embedding_  (None, 5, 256)      1280        ['input_3[0][0]']                
 Layer)                                                                                           
                                                                                                  
 embedding__layer_3 (Embedding_  (None, 6, 256)      1536        ['input_4[0][0]']          

In [8]:
output

<tf.Tensor: shape=(1, 6, 256), dtype=float32, numpy=
array([[[-0.03700479, -0.03185036,  0.00193087, ...,  0.0187241 ,
         -0.04330489,  0.00963604],
        [ 0.01505849, -0.04136921, -0.01855635, ...,  0.02884842,
         -0.04345727, -0.04347426],
        [ 0.04109098,  0.01752318, -0.04535646, ...,  0.03041196,
         -0.0236635 , -0.00302831],
        [ 0.03619131, -0.04778161,  0.00259532, ..., -0.01940738,
          0.02849353,  0.03596172],
        [-0.00900288,  0.01427584, -0.01441203, ..., -0.00231839,
          0.00814171, -0.01818151],
        [ 0.02492912, -0.01239681,  0.00892762, ..., -0.04304081,
         -0.01053611, -0.02696837]]], dtype=float32)>